In [1]:
from sdv.tabular import TVAE, CTGAN

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sdv.evaluation import evaluate
from sdv.metrics.tabular import LinearRegression, MLPRegressor

In [3]:
import pandas as pd
from pathlib import Path

In [5]:
DATASET_DIR = r"/home/thes1067/data/claix_dataset/data/claix"
DATASET_NAME = "claix_posix_npb_4_16_64_nprocs_Ciao_C_1288"
DATASET_PATH = Path(DATASET_DIR, DATASET_NAME).with_suffix(".csv")

In [6]:
df_claix_posix = pd.read_csv(DATASET_PATH)

In [8]:
df_claix_posix.head()

,POSIX_OPENS,POSIX_FILENOS,POSIX_DUPS,POSIX_READS,POSIX_WRITES,POSIX_SEEKS,POSIX_STATS,POSIX_MMAPS,POSIX_FSYNCS,POSIX_FDSYNCS,...,POSIX_F_VARIANCE_RANK_TIME,POSIX_F_VARIANCE_RANK_BYTES,uid,start_time,end_time,nprocs,jobid,lib_ver,hints,bandwidth
0,2288,2106,0,21053,26218,26541,185,-133,0,0,...,1.941565e-06,8.932490e+07,42282,2022-01-29 12:07:26,2022-01-29 12:11:42,24,209310,3.3.1,romio_no_indep_rw=true;cb_nodes=4,4420.630430
1,16112,12718,0,389198,35189,236638,1933,-43,0,0,...,2.783937e+01,4.176090e+16,36826,2021-09-06 14:14:32,2021-09-06 14:15:38,240,23029701,3.1.8,romio_no_indep_rw=true;cb_nodes=4,407.677717
2,646,322,0,5184,161,1458,0,-163,0,0,...,2.515119e-06,0.000000e+00,36826,2021-09-06 11:00:54,2021-09-06 11:00:55,162,23022111,3.1.8,romio_no_indep_rw=true;cb_nodes=4,0.084217
3,2288,2106,0,21053,26218,26541,185,-133,0,0,...,4.719168e-07,8.932490e+07,42282,2022-01-31 14:36:36,2022-01-31 14:40:34,24,175062,3.3.1,romio_no_indep_rw=true;cb_nodes=4,41753.913256
4,2288,2106,0,21053,26218,26541,185,-133,0,0,...,3.402062e-07,8.932490e+07,42282,2022-01-10 11:02:43,2022-01-10 11:06:41,24,217152,3.3.1,romio_no_indep_rw=true;cb_nodes=4,50749.073598


### Drop the non-invariant columns

In [9]:
df_claix_posix = df_claix_posix.drop(['uid', 'jobid', 'hints', 'start_time', 'end_time', 'lib_ver'],
                                                               axis=1)

### Drop columns to match the Blue Waters dataset on which the model was trained

In [10]:
df_claix_posix = df_claix_posix.drop(['POSIX_FDSYNCS',
                                        'POSIX_RENAMED_FROM',
                                        'POSIX_F_VARIANCE_RANK_TIME',
                                        'POSIX_F_VARIANCE_RANK_BYTES'],
                                        axis=1)			

In [11]:
df_claix_posix.head()

,POSIX_OPENS,POSIX_FILENOS,POSIX_DUPS,POSIX_READS,POSIX_WRITES,POSIX_SEEKS,POSIX_STATS,POSIX_MMAPS,POSIX_FSYNCS,POSIX_RENAME_SOURCES,...,POSIX_F_READ_TIME,POSIX_F_WRITE_TIME,POSIX_F_META_TIME,POSIX_TOTAL_TIME,POSIX_F_MAX_READ_TIME,POSIX_F_MAX_WRITE_TIME,POSIX_F_FASTEST_RANK_TIME,POSIX_F_SLOWEST_RANK_TIME,nprocs,bandwidth
0,2288,2106,0,21053,26218,26541,185,-133,0,0,...,0.655522,15.114676,0.004702,15.774899,0.076137,6.184611,0.0,0.046561,24,4420.630430
1,16112,12718,0,389198,35189,236638,1933,-43,0,0,...,2329.200567,100.632465,3.125118,2432.958150,1.931765,0.226163,0.0,17.771790,240,407.677717
2,646,322,0,5184,161,1458,0,-163,0,0,...,44.569253,0.000000,0.437835,45.007087,0.097108,0.000012,0.0,0.281752,162,0.084217
3,2288,2106,0,21053,26218,26541,185,-133,0,0,...,1.637240,0.002378,0.030525,1.670143,0.076095,0.060506,0.0,0.070267,24,41753.913256
4,2288,2106,0,21053,26218,26541,185,-133,0,0,...,1.345281,0.002364,0.026468,1.374114,0.056918,0.056870,0.0,0.058452,24,50749.073598


In [67]:
results = []
for size in [x * 10 for x in range(1, 10)]:
    model = TVAE(batch_size=size, epochs=600)
    model.fit(df_claix_posix)
    synthetic_data = model.sample(num_rows=200)
    score = evaluate(synthetic_data, df_claix_posix)
    lr = LinearRegression.compute(df_claix_posix, synthetic_data, target="bandwidth")
    mlpreg = MLPRegressor.compute(df_claix_posix, synthetic_data, target="bandwidth")

    print(f"Batch size: {size}")
    print(f"LinReg: {lr} MLPReg: {mlpreg} Score: {score}")
    result = {"batch_size":size, "linreg":lr, "mlpreg":mlpreg, "score":score}
    results.append(result)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 10
LinReg: -20.465503791770917 MLPReg: -1.2317694901219758 Score: 0.3930807323619511


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 20
LinReg: -1090.0332796815524 MLPReg: -1.2299711773384927 Score: 0.43190306685079816


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

...

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 80
LinReg: -1709.000839323255 MLPReg: -1.2307979179902073 Score: 0.4463947832333323


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 90
LinReg: -242.61420413673582 MLPReg: -1.2309631493753534 Score: 0.36735350159574093


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [68]:
results

[{'batch_size': 10,
  'linreg': -20.465503791770917,
  'mlpreg': -1.2317694901219758,
  'score': 0.3930807323619511},
 {'batch_size': 20,
  'linreg': -1090.0332796815524,
  'mlpreg': -1.2299711773384927,
  'score': 0.43190306685079816},
 {'batch_size': 30,
  'linreg': -88.57077065364724,
  'mlpreg': -1.2301285018856238,
  'score': 0.4310705541306212},
 {'batch_size': 40,
  'linreg': -129.81439471264952,
  'mlpreg': -1.2307557457128637,
  'score': 0.4395939184183522},
 {'batch_size': 50,
  'linreg': -650.6506904156969,
  'mlpreg': -1.2305683347128835,
  'score': 0.42270995912345133},
 {'batch_size': 60,
  'linreg': -335.2858235084207,
  'mlpreg': -1.230314385480186,
  'score': 0.4190268483656428},
 {'batch_size': 70,
  'linreg': -134.7270777952015,
  'mlpreg': -1.2300771638460741,
  'score': 0.4264118201809118},
 {'batch_size': 80,
  'linreg': -1709.000839323255,
  'mlpreg': -1.2307979179902073,
  'score': 0.4463947832333323},
 {'batch_size': 90,
  'linreg': -242.61420413673582,
  'mlpr

In [66]:
model = TVAE(batch_size=70, epochs=900)

In [67]:
model.fit(df_claix_posix)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

In [68]:
synthetic_data = model.sample(num_rows=200)

In [69]:
synthetic_data.head()

,POSIX_OPENS,POSIX_FILENOS,POSIX_DUPS,POSIX_READS,POSIX_WRITES,POSIX_SEEKS,POSIX_STATS,POSIX_MMAPS,POSIX_FSYNCS,POSIX_RENAME_SOURCES,...,POSIX_F_READ_TIME,POSIX_F_WRITE_TIME,POSIX_F_META_TIME,POSIX_TOTAL_TIME,POSIX_F_MAX_READ_TIME,POSIX_F_MAX_WRITE_TIME,POSIX_F_FASTEST_RANK_TIME,POSIX_F_SLOWEST_RANK_TIME,nprocs,bandwidth
0,2222,1923,0,20079,26204,19044,179,-133,0,0,...,0.492313,1.292794,0.002993,1.111525,0.005292,0.000012,0.000000,0.387600,24,45934.319126
1,10079,8409,0,261345,25066,132396,1310,-33,0,0,...,580.402486,247.115538,58.317101,1440.891305,1.591553,0.338043,0.000000,35.476365,170,0.084217
2,2141,2263,0,21213,26154,29012,208,-138,0,0,...,35.723524,0.000000,0.002993,89.069662,0.136489,0.140537,0.000007,0.714222,26,1042.684649
3,2217,2254,0,20913,26227,33105,205,-133,0,0,...,8.568469,7.359427,14.596811,120.975363,0.008830,0.179200,0.000036,0.946820,25,47983.228714
4,2248,1921,0,21903,26036,28118,171,-133,0,0,...,10.152191,0.000000,0.002993,78.011767,0.118865,0.225067,0.000000,1.821413,24,61493.339148


In [70]:
evaluate(synthetic_data, df_claix_posix)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.4425423143655296

In [71]:
evaluate(synthetic_data, df_claix_posix, aggregate=False)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,BNLogLikelihood,BayesianNetwork Log Likelihood,NaN,NaN,-inf,0.0,MAXIMIZE,Please install pomegranate with `pip install p...
1,LogisticDetection,LogisticRegression Detection,4.869189e-01,0.486919,0.0,1.0,MAXIMIZE,None
2,SVCDetection,SVC Detection,3.424141e-01,0.342414,0.0,1.0,MAXIMIZE,None
3,BinaryDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
4,BinaryAdaBoostClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
5,BinaryLogisticRegression,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
6,BinaryMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
7,MulticlassDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
8,MulticlassMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
9,LinearRegression,None,NaN,NaN,-inf,1.0,MAXIMIZE,`target` must be passed either directly or ins...


In [72]:
LinearRegression.compute(df_claix_posix, synthetic_data, target="bandwidth")

-96.10543726880951

In [73]:
MLPRegressor.compute(df_claix_posix, synthetic_data, target="bandwidth")

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


-1.2306959547377372

In [12]:
model = TVAE(batch_size=10, epochs=900)

In [13]:
model.fit(df_claix_posix)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

In [14]:
synthetic_data = model.sample(num_rows=200)

In [15]:
synthetic_data.head()

,POSIX_OPENS,POSIX_FILENOS,POSIX_DUPS,POSIX_READS,POSIX_WRITES,POSIX_SEEKS,POSIX_STATS,POSIX_MMAPS,POSIX_FSYNCS,POSIX_RENAME_SOURCES,...,POSIX_F_READ_TIME,POSIX_F_WRITE_TIME,POSIX_F_META_TIME,POSIX_TOTAL_TIME,POSIX_F_MAX_READ_TIME,POSIX_F_MAX_WRITE_TIME,POSIX_F_FASTEST_RANK_TIME,POSIX_F_SLOWEST_RANK_TIME,nprocs,bandwidth
0,2449,2105,0,179843,25549,24297,145,-134,0,0,...,36.117835,11.577302,1.097582,65.239438,0.103596,0.004291,0.000000,47.831330,22,308.920136
1,2226,2200,0,12654,25638,22222,235,-134,0,0,...,17.521766,17.684855,15.212835,132.194907,0.165857,0.083080,0.000022,37.651915,23,11530.271577
2,2340,2344,0,17440,24682,27653,159,-134,0,0,...,46.998483,0.000000,0.002993,1.111525,0.127493,0.567201,0.000000,3.586568,23,3936.530147
3,164,0,0,4390,638,10577,39,-13,0,0,...,8.367738,25.819574,31.612727,113.823209,0.005292,0.000012,0.000472,2.860517,16,11397.620631
4,2296,2182,0,16715,26406,26955,200,-133,0,0,...,4.450937,11.449233,14.972485,134.227703,0.171091,0.072690,0.000181,0.004909,24,52825.497940


In [16]:
evaluate(synthetic_data, df_claix_posix)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.429445715648654

In [17]:
evaluate(synthetic_data, df_claix_posix, aggregate=False)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,BNLogLikelihood,BayesianNetwork Log Likelihood,NaN,NaN,-inf,0.0,MAXIMIZE,Please install pomegranate with `pip install p...
1,LogisticDetection,LogisticRegression Detection,3.462871e-01,0.346287,0.0,1.0,MAXIMIZE,None
2,SVCDetection,SVC Detection,2.997047e-01,0.299705,0.0,1.0,MAXIMIZE,None
3,BinaryDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
4,BinaryAdaBoostClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
5,BinaryLogisticRegression,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
6,BinaryMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
7,MulticlassDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
8,MulticlassMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
9,LinearRegression,None,NaN,NaN,-inf,1.0,MAXIMIZE,`target` must be passed either directly or ins...


In [18]:
LinearRegression.compute(df_claix_posix, synthetic_data, target="bandwidth")

-202.17362311370889

In [19]:
MLPRegressor.compute(df_claix_posix, synthetic_data, target="bandwidth")

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


-1.2306462048356615

In [74]:
synthetic_data.to_csv(DATASET_DIR+"/posix_synth_TVAE_70_batch_900_epochs.csv", index=False)

### CTGAN

In [22]:
results = []
for size in [x * 10 for x in range(1, 10)]:
    model = CTGAN(batch_size=size, epochs=600)
    model.fit(df_claix_posix)
    synthetic_data = model.sample(num_rows=200)
    score = evaluate(synthetic_data, df_claix_posix)
    lr = LinearRegression.compute(df_claix_posix, synthetic_data, target="bandwidth")
    mlpreg = MLPRegressor.compute(df_claix_posix, synthetic_data, target="bandwidth")

    print(f"Batch size: {size}")
    print(f"LinReg: {lr} MLPReg: {mlpreg} Score: {score}")
    result = {"batch_size":size, "linreg":lr, "mlpreg":mlpreg, "score":score}
    results.append(result)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 10
LinReg: -7.3941372491390585 MLPReg: -1.551991653772045 Score: 0.34947020382534727


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 20
LinReg: -1.8489023083416636 MLPReg: -1.3904047602959992 Score: 0.4339821024691371


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

...

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 80
LinReg: -7.6892351813528865 MLPReg: -1.508753800963873 Score: 0.33247437270998975


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

Batch size: 90
LinReg: -1.7833851743624458 MLPReg: -1.2898471808784375 Score: 0.37700266148398026


/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


In [23]:
results

[{'batch_size': 10,
  'linreg': -7.3941372491390585,
  'mlpreg': -1.551991653772045,
  'score': 0.34947020382534727},
 {'batch_size': 20,
  'linreg': -1.8489023083416636,
  'mlpreg': -1.3904047602959992,
  'score': 0.4339821024691371},
 {'batch_size': 30,
  'linreg': -0.011339387166230619,
  'mlpreg': -1.2558381008598039,
  'score': 0.35209452926473556},
 {'batch_size': 40,
  'linreg': -0.22552042369592007,
  'mlpreg': -26438.067089229862,
  'score': 0.33385745929887894},
 {'batch_size': 50,
  'linreg': -4.189479960163821,
  'mlpreg': -66115.43423338307,
  'score': 0.30227919461521763},
 {'batch_size': 60,
  'linreg': -2.5136808882385204,
  'mlpreg': -395.1653963115222,
  'score': 0.3408996721300037},
 {'batch_size': 70,
  'linreg': -3.4164391236320126,
  'mlpreg': -43.52454731881742,
  'score': 0.3511167338132304},
 {'batch_size': 80,
  'linreg': -7.6892351813528865,
  'mlpreg': -1.508753800963873,
  'score': 0.33247437270998975},
 {'batch_size': 90,
  'linreg': -1.7833851743624458,
 

In [33]:
model = CTGAN(batch_size=20, epochs=1200)

In [34]:
model.fit(df_claix_posix)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/mixture/_base.py:131: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  cluster.KMeans(
/home/eo0805

In [35]:
synthetic_data_ctgan = model.sample(num_rows=200)

In [36]:
synthetic_data_ctgan.head()

,POSIX_OPENS,POSIX_FILENOS,POSIX_DUPS,POSIX_READS,POSIX_WRITES,POSIX_SEEKS,POSIX_STATS,POSIX_MMAPS,POSIX_FSYNCS,POSIX_RENAME_SOURCES,...,POSIX_F_READ_TIME,POSIX_F_WRITE_TIME,POSIX_F_META_TIME,POSIX_TOTAL_TIME,POSIX_F_MAX_READ_TIME,POSIX_F_MAX_WRITE_TIME,POSIX_F_FASTEST_RANK_TIME,POSIX_F_SLOWEST_RANK_TIME,nprocs,bandwidth
0,8,0,0,440,28591,54941,426,-130,0,0,...,0.492313,0.000000,264.874974,1.111525,0.038796,3.344580,0.000418,17.360589,58,170.558175
1,2167,1576,0,15665,28720,4504,47,-103,0,0,...,0.492313,79.543937,224.347285,1.111525,0.588375,9.315238,0.000000,0.004909,44,59348.134361
2,8,820,0,2585,24365,743,262,-1,0,0,...,79.164004,0.000000,0.002993,75.296271,2.268877,0.000012,0.000000,3.335360,53,11162.062565
3,1533,1216,0,19097,28633,38236,169,-123,0,0,...,27.588697,74.313010,0.002993,1.111525,0.704903,2.252892,0.000000,11.287408,30,17197.956099
4,557,1584,0,440,28157,26870,121,-106,0,0,...,0.492313,27.815412,0.002993,230.336321,0.492760,0.000012,0.000977,9.559771,23,49761.785092


In [28]:
evaluate(synthetic_data_ctgan, df_claix_posix)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

0.3613281275942319

In [37]:
evaluate(synthetic_data_ctgan, df_claix_posix, aggregate=False)

/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/eo080593/Software/miniconda3/envs/sdv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,BNLogLikelihood,BayesianNetwork Log Likelihood,NaN,NaN,-inf,0.0,MAXIMIZE,Please install pomegranate with `pip install p...
1,LogisticDetection,LogisticRegression Detection,2.661138e-01,0.266114,0.0,1.0,MAXIMIZE,None
2,SVCDetection,SVC Detection,8.144760e-01,0.814476,0.0,1.0,MAXIMIZE,None
3,BinaryDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
4,BinaryAdaBoostClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
5,BinaryLogisticRegression,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
6,BinaryMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
7,MulticlassDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
8,MulticlassMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
9,LinearRegression,None,NaN,NaN,-inf,1.0,MAXIMIZE,`target` must be passed either directly or ins...


In [38]:
LinearRegression.compute(df_claix_posix, synthetic_data_ctgan, target="bandwidth")

-0.037000942099560064

In [39]:
MLPRegressor.compute(df_claix_posix, synthetic_data_ctgan, target="bandwidth")

-16682.39320261229

In [40]:
synthetic_data_ctgan.to_csv(DATASET_DIR+"/posix_synth_CTGAN_20_batch_1200_epochs.csv", index=False)